In [4]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import utils

### Prepare data

In [ ]:
pretrained_embeddings_model_name = 'ml_projects_vector_embeddings_model'
custom_embeddings_model_name = 'ml_projects_custom_embeddings_model'

pipeline_name = 'pipeline'

In [5]:
with open('config.json', 'r') as f:
    dataset_name = json.load(f)['dataset_name']

data = utils.load_bids_df(dataset_name)
data['bid_description'] = data['bid_description'].fillna('')

In [6]:
X_1, X_2 = data['project_description'].values, data['bid_description'].values
y = data['bid_award_status'].values

pos_weight = len(y) / sum(y)
sample_weight = y * pos_weight + 1.

In [7]:
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import StandardScaler

cat_columns = ['bid_quality', 'bid_communication', 'bid_professionalism']

k_bins = KBinsDiscretizer(2, encode='ordinal',strategy='uniform', subsample=None)
data_cat = k_bins.fit_transform(data[cat_columns]).astype(bool)

for i, col in enumerate(cat_columns):
    data[col] = data_cat[:, i]

df_prepared = utils.prepare_df(data)
df_prepared = df_prepared.drop(['project_id', 'bid_award_status'], axis=1)

cols_num = df_prepared.select_dtypes(include=['number']).columns
cols_cat = df_prepared.select_dtypes(exclude=['number']).columns

scaler = StandardScaler()
X_num = scaler.fit_transform(df_prepared[cols_num])

X_3 = np.append(X_num, df_prepared[cols_cat], axis=1).astype('float32')

#### basic evaluation functions

In [ ]:
def evaluate_model(model, test_set):
    metrics = model.evaluate(test_set)
    metrics = np.array(metrics)
    metrics[1:] *= 100
    print("loss: {:.2f}, accuracy: {:.2f}%, precision: {:.2f}%, recall: {:.2f}%".format(*metrics))

In [ ]:
def get_accuracies(y,pred,project_ids,to=2):
    top_accuracy, top_2_accuracy = [],[]
    for project_id in set(project_ids):
        indexes = project_ids == project_id

        y_project,pred_project = y[indexes].reshape(-1), pred[indexes].reshape(-1)

        if sum(y_project) == 0 or len(y_project) < to or sum(pred_project) == 0: continue

        # choosing the best bid (along predictions)
        sorted_pred = np.argsort(pred_project)[:-to -1:-1]
        top_accuracy.append( y_project[sorted_pred[0]] )

        # choosing the top 2 best bids (along predictions)
        top_2_accuracy.append( any(y_project[ sorted_pred ]) )

    top_accuracy = np.mean(top_accuracy) * 100
    top_2_accuracy = np.mean(top_2_accuracy) * 100

    return top_accuracy, top_2_accuracy

### Train model

In [ ]:
ds = tf.data.Dataset.from_tensor_slices(((X_1, X_2, ), y, sample_weight))

train_size,val_size = int(.7*len(data)), int(.15*len(data))

train_ds = ds.take(train_size).shuffle(1000).batch(32).prefetch(1)
val_ds = ds.skip(train_size)

test_ds = val_ds.skip(val_size).batch(32).prefetch(1)
val_ds = val_ds.take(val_size).batch(32).prefetch(1)

In [ ]:
import tensorflow_hub as hub

embed_1 = keras.models.Sequential([
  hub.KerasLayer('https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1',
                 dtype=tf.string, input_shape=[], output_shape=[128],
                 name='embed_1', trainable=False),
])
embed_2 = keras.models.Sequential([
  hub.KerasLayer('https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1',
                 dtype=tf.string, input_shape=[], output_shape=[128],
                 name='embed_2', trainable=False),
])

In [ ]:
concat = keras.layers.Concatenate()([embed_1.output, embed_2.output])

x = keras.layers.Dropout(.2)(concat)

x = keras.layers.Dense(32, activation='relu')(x)
x = keras.layers.Dense(16, activation='relu')(x)
outputs = keras.layers.Dense(1, activation='sigmoid')(x)

model = keras.models.Model(inputs=[embed_1.input, embed_2.input,],
                           outputs=[outputs])

model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=1e-3),
              metrics=['accuracy', keras.metrics.Precision(),
                       keras.metrics.Recall()], weighted_metrics=[])

early_stop = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=16,
                                           restore_best_weights=True)
lr_scheduler = keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy',
                                                 patience=3, factor=.8)

In [ ]:
model.fit(train_ds, epochs=100, validation_data=val_ds,
          callbacks=[early_stop])

Epoch 1/100
210/210 [==============================] - 14s 28ms/step - loss: 1.3649 - accuracy: 0.5488 - precision: 0.0900 - recall: 0.4936 - val_loss: 1.4259 - val_accuracy: 0.1933 - val_precision: 0.0931 - val_recall: 0.8797
Epoch 2/100
210/210 [==============================] - 1s 7ms/step - loss: 1.3336 - accuracy: 0.4492 - precision: 0.1006 - recall: 0.7195 - val_loss: 1.4196 - val_accuracy: 0.1214 - val_precision: 0.0950 - val_recall: 0.9925
Epoch 3/100
210/210 [==============================] - 3s 12ms/step - loss: 1.3161 - accuracy: 0.4533 - precision: 0.1030 - recall: 0.7341 - val_loss: 1.4094 - val_accuracy: 0.4445 - val_precision: 0.1123 - val_recall: 0.7218
Epoch 4/100
210/210 [==============================] - 5s 23ms/step - loss: 1.2993 - accuracy: 0.4824 - precision: 0.1075 - recall: 0.7268 - val_loss: 1.4515 - val_accuracy: 0.6253 - val_precision: 0.1203 - val_recall: 0.4812
Epoch 5/100
210/210 [==============================] - 2s 11ms/step - loss: 1.2712 - accuracy: 0

In [ ]:
evaluate_model(model, test_ds)

45/45 [==============================] - 0s 7ms/step - loss: 1.6914 - accuracy: 0.7199 - precision: 0.1188 - recall: 0.3413
loss: 1.69, accuracy: 71.99%, precision: 11.88%, recall: 34.13%


In [ ]:
test_projects = data.iloc[train_size+val_size:].reset_index(drop=True)

pred = model.predict(test_ds)

get_accuracies(test_projects['bid_award_status'].values, pred,
               test_projects['project_id'].values) # top 1 and top 2

45/45 [==============================] - 0s 9ms/step


(16.129032258064516, 29.838709677419356)

In [ ]:
get_accuracies(test_projects['bid_award_status'].values, pred,
               test_projects['project_id'].values, to=3) # top 1 and top 3

(16.129032258064516, 42.74193548387097)

#### Combined model

In [ ]:
ds = tf.data.Dataset.from_tensor_slices(((X_1, X_2, X_3), y, sample_weight))

train_size,val_size = int(.7*len(data)), int(.15*len(data))

train_ds = ds.take(train_size).shuffle(1000).batch(32).prefetch(1)
val_ds = ds.skip(train_size)

test_ds = val_ds.skip(val_size).batch(32).prefetch(1)
val_ds = val_ds.take(val_size).batch(32).prefetch(1)

In [ ]:
model.trainable = False

input_2 = keras.layers.Input(shape=[14])

concat = keras.layers.Concatenate()([model.output, input_2])

x = keras.layers.Dense(8)(concat)
output = keras.layers.Dense(1, activation='sigmoid')(x)

combined_model = keras.models.Model(inputs=[embed_1.input, embed_2.input, input_2],
                                    outputs=[output])

In [ ]:
combined_model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=1e-3),
              metrics=['accuracy', keras.metrics.Precision(),
                       keras.metrics.Recall()], weighted_metrics=[])

early_stop = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=16,
                                           restore_best_weights=True)
lr_scheduler = keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy',
                                                 patience=3, factor=.8)

In [ ]:
combined_model.fit(train_ds, epochs=100, validation_data=val_ds,
          callbacks=[early_stop])

Epoch 1/100
210/210 [==============================] - 5s 19ms/step - loss: 1.3451 - accuracy: 0.5446 - precision_1: 0.1121 - recall_1: 0.6576 - val_loss: 1.4017 - val_accuracy: 0.3329 - val_precision_1: 0.1077 - val_recall_1: 0.8496
Epoch 2/100
210/210 [==============================] - 4s 21ms/step - loss: 1.1757 - accuracy: 0.5832 - precision_1: 0.1533 - recall_1: 0.9016 - val_loss: 1.3724 - val_accuracy: 0.4473 - val_precision_1: 0.1251 - val_recall_1: 0.8271
Epoch 3/100
210/210 [==============================] - 3s 17ms/step - loss: 1.0848 - accuracy: 0.6476 - precision_1: 0.1790 - recall_1: 0.9180 - val_loss: 1.3791 - val_accuracy: 0.5597 - val_precision_1: 0.1349 - val_recall_1: 0.6917
Epoch 4/100
210/210 [==============================] - 3s 16ms/step - loss: 1.0048 - accuracy: 0.7031 - precision_1: 0.2047 - recall_1: 0.9071 - val_loss: 1.3770 - val_accuracy: 0.5743 - val_precision_1: 0.1392 - val_recall_1: 0.6917
Epoch 5/100
210/210 [==============================] - 3s 16ms/s

In [ ]:
evaluate_model(combined_model, test_ds)

45/45 [==============================] - 0s 4ms/step - loss: 1.8276 - accuracy: 0.7686 - precision_1: 0.1589 - recall_1: 0.3810
loss: 1.83, accuracy: 76.86%, precision: 15.89%, recall: 38.10%


In [ ]:
test_projects = data.iloc[train_size+val_size:].reset_index(drop=True)

pred = combined_model.predict(test_ds)

get_accuracies(test_projects['bid_award_status'].values, pred,
               test_projects['project_id'].values) # top 1 and top 2

45/45 [==============================] - 0s 3ms/step


(21.774193548387096, 35.483870967741936)

In [ ]:
get_accuracies(test_projects['bid_award_status'].values, pred,
               test_projects['project_id'].values, to=3) # top 1 and top 3

(21.774193548387096, 52.41935483870967)

### Saving model and pipeline

In [ ]:
combined_model.save(f'models/{pretrained_embeddings_model_name}.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

full_pipeline = ColumnTransformer([
    ('scaler', scaler, cols_num),
    ('sealed', 'passthrough', ['bid_sealed']),
    ('cat_pipeline', k_bins, cat_columns),
    ('greetings', 'passthrough', ['bid_greetings'])
])
full_pipeline = full_pipeline.fit(df_prepared)

In [ ]:
full_pipeline.transform(df_prepared).astype('float32') == X_3 # for the whole dataset

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [ ]:
import joblib

joblib.dump(full_pipeline, f'models/{pipeline_name}.json')

['models/pipeline.json']

### Custom embeddings

In [ ]:
ds = tf.data.Dataset.from_tensor_slices(((X_1, X_2), y, sample_weight))

train_size,val_size = int(.7*len(data)), int(.15*len(data))

train_ds = ds.take(train_size)
val_ds = ds.skip(train_size)

test_ds = val_ds.skip(val_size)
val_ds = val_ds.take(val_size)

In [ ]:
def preprocess_batch(X_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)
    X_batch = tf.strings.regex_replace(X_batch, rb"<br\s*/?>", b" ")
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
    X_batch = tf.strings.split(X_batch)

    return X_batch.to_tensor(default_value=b"<pad>")

def preprocess(X_batch, y_batch, sample_weight):

    X_1 = preprocess_batch(X_batch[0])
    X_2 = preprocess_batch(X_batch[1])

    return (X_1, X_2, *X_batch[2:]), y_batch, sample_weight

In [ ]:
from collections import Counter

vocabulary = Counter()
for X_batch, y_batch, _ in ds.batch(32).map(preprocess):

    for project_description in X_batch[0]:
        vocabulary.update(list(project_description.numpy()))
    for bid_description in X_batch[1]:
        vocabulary.update(list(bid_description.numpy()))

In [ ]:
vocab_size = 10000
truncated_vocabulary = [
    word for word, count in vocabulary.most_common()[:vocab_size]]

In [ ]:
word_to_id = {word: index for index, word in enumerate(truncated_vocabulary)}
for word in b"I think it's impoossible".split():
    print(word_to_id.get(word) or vocab_size)

1
348
784
10000


In [ ]:
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [ ]:
table.lookup(tf.constant([b"I think it's impoossible".split()]))

<tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[    1,   348,   784, 10737]])>

In [ ]:
pad_token = table.lookup(tf.constant([b"<pad>".split()]))[0][0]

def encode_words(X_batch, y_batch, sample_weight):
  X_1 = table.lookup(X_batch[0])
  X_2 = table.lookup(X_batch[1])

  return (X_1,X_2, *X_batch[2:]), y_batch, sample_weight

train_set = train_ds.batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

In [ ]:
val_set = val_ds.batch(32).map(preprocess).map(encode_words).prefetch(1)
test_set = test_ds.batch(32).map(preprocess).map(encode_words).prefetch(1)

In [ ]:
embed_size = 64
embed_1 = keras.models.Sequential([
    keras.layers.Embedding(vocab_size+num_oov_buckets, embed_size,
                                 mask_zero=True, input_shape=[None]),
])

embed_2 = keras.models.Sequential([
    keras.layers.Embedding(vocab_size+num_oov_buckets, embed_size,
                                 mask_zero=True, input_shape=[None]),
])

concat = keras.layers.Concatenate(axis=1,
                                  name="concat_1")([embed_1.output, embed_2.output])

x = keras.layers.GRU(128, return_sequences=True, name='gru_1.2', recurrent_dropout=.3)(concat)
x = keras.layers.GRU(64, name='gru_2', recurrent_dropout=.3)(x)

output = keras.layers.Dense(1, activation='sigmoid')(x)

model = keras.models.Model(inputs=[embed_1.input, embed_2.input], outputs=[output])

model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()],
              weighted_metrics=[])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 embedding_input (InputLaye  [(None, None)]               0         []                            
 r)                                                                                               
                                                                                                  
 embedding_1_input (InputLa  [(None, None)]               0         []                            
 yer)                                                                                             
                                                                                                  
 embedding (Embedding)       (None, None, 64)             704000    ['embedding_input[0][0]']     
                                                                                              

In [ ]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10,
                                           restore_best_weights=True)

model.fit(train_set, validation_data=val_set, epochs=6, callbacks=[early_stop])

Epoch 1/6
210/210 [==============================] - 242s 1s/step - loss: 1.3404 - accuracy: 0.3785 - precision: 0.0933 - recall: 0.7541 - val_loss: 1.3826 - val_accuracy: 0.1940 - val_precision: 0.0976 - val_recall: 0.9323
Epoch 2/6
210/210 [==============================] - 197s 934ms/step - loss: 1.0321 - accuracy: 0.6992 - precision: 0.1883 - recall: 0.8051 - val_loss: 1.4550 - val_accuracy: 0.6539 - val_precision: 0.1318 - val_recall: 0.4887
Epoch 3/6
210/210 [==============================] - 193s 920ms/step - loss: 0.7595 - accuracy: 0.8000 - precision: 0.2747 - recall: 0.8761 - val_loss: 1.9163 - val_accuracy: 0.6371 - val_precision: 0.1257 - val_recall: 0.4887
Epoch 4/6
210/210 [==============================] - 193s 920ms/step - loss: 0.6293 - accuracy: 0.8303 - precision: 0.3148 - recall: 0.9071 - val_loss: 2.1160 - val_accuracy: 0.5269 - val_precision: 0.1146 - val_recall: 0.6090
Epoch 5/6
210/210 [==============================] - 193s 922ms/step - loss: 0.5589 - accuracy:

In [ ]:
evaluate_model(model, test_set)

45/45 [==============================] - 3s 68ms/step - loss: 2.2980 - accuracy: 0.6341 - precision: 0.1141 - recall: 0.4683
loss: 2.30, accuracy: 63.41%, precision: 11.41%, recall: 46.83%


In [ ]:
test_projects = data.iloc[train_size+val_size:].reset_index(drop=True)

pred = model.predict(test_set)

get_accuracies(test_projects['bid_award_status'].values, pred,
               test_projects['project_id'].values) # top 1 and top 2

45/45 [==============================] - 3s 68ms/step


(25.0, 35.483870967741936)

In [ ]:
get_accuracies(test_projects['bid_award_status'].values, pred,
               test_projects['project_id'].values, to=3) # top 1 and top 3

(25.0, 45.96774193548387)

In [ ]:
ds = tf.data.Dataset.from_tensor_slices(((X_1, X_2, X_3), y, sample_weight))

train_size,val_size = int(.7*len(data)), int(.15*len(data))

train_ds = ds.take(train_size)
val_ds = ds.skip(train_size)

test_ds = val_ds.skip(val_size)
val_ds = val_ds.take(val_size)

train_set = train_ds.batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

val_set = val_ds.batch(32).map(preprocess).map(encode_words).prefetch(1)
test_set = test_ds.batch(32).map(preprocess).map(encode_words).prefetch(1)

In [ ]:
model.trainable = False

input_2 = keras.layers.Input(shape=[14])

concat = keras.layers.Concatenate()([model.output, input_2])

x = keras.layers.Dense(8)(concat)
output = keras.layers.Dense(1, activation='sigmoid')(x)

combined_model = keras.models.Model(inputs=[embed_1.input, embed_2.input, input_2],
                                    outputs=[output])

In [ ]:
combined_model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=1e-4),
              metrics=['accuracy', keras.metrics.Precision(),
                       keras.metrics.Recall()], weighted_metrics=[])

In [ ]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10,
                                           restore_best_weights=True)

combined_model.fit(train_set, validation_data=val_set, epochs=5,
                   callbacks=[early_stop])

Epoch 1/5
210/210 [==============================] - 118s 540ms/step - loss: 2.0664 - accuracy: 0.4675 - precision_1: 0.0851 - recall_1: 0.5628 - val_loss: 2.3394 - val_accuracy: 0.5206 - val_precision_1: 0.0914 - val_recall_1: 0.4662
Epoch 2/5
210/210 [==============================] - 126s 603ms/step - loss: 1.9707 - accuracy: 0.4475 - precision_1: 0.0843 - recall_1: 0.5811 - val_loss: 2.2154 - val_accuracy: 0.5080 - val_precision_1: 0.0903 - val_recall_1: 0.4737
Epoch 3/5
210/210 [==============================] - 120s 570ms/step - loss: 1.8890 - accuracy: 0.4346 - precision_1: 0.0850 - recall_1: 0.6029 - val_loss: 2.1108 - val_accuracy: 0.4920 - val_precision_1: 0.0908 - val_recall_1: 0.4962
Epoch 4/5
210/210 [==============================] - 111s 529ms/step - loss: 1.8169 - accuracy: 0.4259 - precision_1: 0.0860 - recall_1: 0.6230 - val_loss: 2.0214 - val_accuracy: 0.4871 - val_precision_1: 0.0954 - val_recall_1: 0.5338
Epoch 5/5
210/210 [==============================] - 116s 55

In [ ]:
evaluate_model(combined_model, test_set)

45/45 [==============================] - 8s 179ms/step - loss: 1.8944 - accuracy: 0.5352 - precision_1: 0.0945 - recall_1: 0.5000
loss: 1.89, accuracy: 53.52%, precision: 9.45%, recall: 50.00%


In [ ]:
test_projects = data.iloc[train_size+val_size:].reset_index(drop=True)

pred = combined_model.predict(test_set)

get_accuracies(test_projects['bid_award_status'].values, pred,
               test_projects['project_id'].values) # top 1 and top 2

45/45 [==============================] - 5s 94ms/step


(16.129032258064516, 27.419354838709676)

In [ ]:
get_accuracies(test_projects['bid_award_status'].values, pred,
               test_projects['project_id'].values, to=3) # top 1 and top 2

(16.129032258064516, 39.516129032258064)

### Saving model and pipeline

In [ ]:
combined_model.save(f'models/{custom_embeddings_model_name}.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

full_pipeline = ColumnTransformer([
    ('scaler', scaler, cols_num),
    ('sealed', 'passthrough', ['bid_sealed']),
    ('cat_pipeline', k_bins, cat_columns),
    ('greetings', 'passthrough', ['bid_greetings'])
])
full_pipeline = full_pipeline.fit(df_prepared)

In [ ]:
import joblib

joblib.dump(full_pipeline, f'models/{pipeline_name}.json')

['models/pipeline.json']

In [ ]:
import json

with open('models/vocabulary.json', 'w') as f:
    json.dump([str(elem) for elem in truncated_vocabulary], f)